# Разархивирование файлов

In [1]:
import os
import re
from subprocess import check_call
from subprocess import CalledProcessError
import py7zr

### Шаг 1. Прописываю классы разархивации

In [2]:
# Получим список папок

PATH_TO_STORAGE = '/home/eugene/Projects/loader/storage/docs'


class Customer:
    
    def __init__(self, customer_inn:str = '7722765428'):
        self.customer_inn = customer_inn
        self.projects_list = [os.path.join(PATH_TO_STORAGE, customer_inn, i) for i in os.listdir(os.path.join(PATH_TO_STORAGE, customer_inn))]
        
    def __str__(self):
        # return f"<Customer inn {self.customer_inn}, {len(self.projects_list)} projects>"
        return self.projects_list

In [3]:
class Dearchivator:
    
    def __init__(self, path_to_project):
        self.path_to_project = path_to_project
        self.archives = sorted([os.path.join(self.path_to_project, i) \
                         for i in os.listdir(self.path_to_project) \
                         if i.endswith(('.rar', '.zip', '.7z'))])
        self.to_unzip = []
        unrared = []
        for i in self.archives:
            pattern = re.compile('.part[0-9]+\.[rarzip7]+')
            if pattern.search(i):
                if i.split('.part')[0] not in unrared:
                    # send i to unzipping
                    self.to_unzip.append(i)
                    unrared.append(i.split('.part')[0])
            else:
                # send i to unzipping
                self.to_unzip.append(i)
    
    def unzip(self):
        for i in self.to_unzip:
            if i.endswith('.rar'):
                try:
                    check_call(["unrar","e","-o-", i, self.path_to_project])
                except CalledProcessError as cpe:
                    print("Error on ", self.path_to_project)
            if i.endswith('.zip'):
                try:
                    check_call(["unzip", "-j", "-n", i, "-d", self.path_to_project])
                except CalledProcessError as cpe:
                    print("Error on ", self.path_to_project)
            if i.endswith('.7z'):
                try:
                    check_call(["7z", "e", "-y", i, f"-o{self.path_to_project}"])
                except CalledProcessError as cpe:
                    print("Error on ", self.path_to_project)

        return True
    
    def _delete_archives(self):
        for i in self.archives:
            os.remove(i)
    
    def __str__(self):
        return self.archives

### Шаг 2. Разархивирую проекты

In [4]:
# Нахожу все папку Заказчика и все проекты ы ней

customer_inn = '7728381587'
customer = Customer(customer_inn=customer_inn)
customers_projects = customer.__str__()

# Первые 10 из них
print(customers_projects[:10])

# Посмотрим сколько всего их
len(customers_projects)

['/home/eugene/Projects/loader/storage/docs/7728381587/0173200001420000073', '/home/eugene/Projects/loader/storage/docs/7728381587/0173200001420000711', '/home/eugene/Projects/loader/storage/docs/7728381587/0373200008518000096', '/home/eugene/Projects/loader/storage/docs/7728381587/0173200001420000889', '/home/eugene/Projects/loader/storage/docs/7728381587/0173200001419001944', '/home/eugene/Projects/loader/storage/docs/7728381587/0173200001419000077', '/home/eugene/Projects/loader/storage/docs/7728381587/0173200001420000791', '/home/eugene/Projects/loader/storage/docs/7728381587/0173200001420000345', '/home/eugene/Projects/loader/storage/docs/7728381587/0173200001420001275', '/home/eugene/Projects/loader/storage/docs/7728381587/0173200001418000707']


93

In [ ]:
for i in customers_projects:
    dearchivator = Dearchivator(i)
    if dearchivator.unzip():
        dearchivator._delete_archives()